In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

##### **I will be training a ResNet34 model on the Fish Dataset using Pytorch. This notebook is in an explanatory form so it will be easy for everyone to understand.**

# Imports and downloads

In [ ]:
import torch
import torchvision
from torchvision import models
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F

import tarfile
from PIL import Image
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import train_test_split

import albumentations as A

from albumentations.pytorch import ToTensorV2

from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install albumentations==0.4.6

# Data Cleaning
#### We will first try to separate the images from the GT images in the folders for each class and use their titles to create our labels.

In [ ]:
root_dir = '../input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset'
path = Path(root_dir)
path_images = list(path.glob('**/*.png'))

images_paths = [str(path_image) for path_image in path_images if 'GT' not in str(path_image)]
print(f'Number of training images :{len(images_paths)}')

labels = [os.path.split(os.path.split(name)[0])[1] for name in images_paths]
print(f'Number of labels :{len(labels)}')

#### The labels must be converted into numbers to make it easier to work with for our model.

In [ ]:
classes = list(set(labels))
labels_dict = {label : i for i,label in enumerate(classes)}
labels_val = [labels_dict[label_key] for label_key in labels]

#### Lets see the number of classes and what they are.

In [ ]:
print("No. of classes:", len(classes))
print("Various Classes:", classes)

# Generating Train, Test and Validation Datasets

#### Let us split the data first...

In [ ]:
random_seed = 42
torch.manual_seed(random_seed);

In [ ]:
#Splitting test data from the whole dataset
data, test_data, labels, test_labels = train_test_split(images_paths, labels_val, test_size=0.15, shuffle=True)
#Splitting train data and validation data
train_data, val_data, train_labels, val_labels = train_test_split(data, labels, test_size=0.1, shuffle=True)

#### The class `FishDataset()` below will be useful in creating the datasets further down the line.

In [ ]:
class FishDataset(torch.utils.data.Dataset):
    def __init__(self, images: list, labels: list, transform=None):
        super().__init__()
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self, ):
        return len(self.labels)

    def __getitem__(self, index):
        input_image = self.images[index]
        label = self.labels[index]
        image = np.array(Image.open(input_image).convert("RGB"))

        if self.transform is not None:
            augmentations = self.transform(image=image)
            image = augmentations["image"]

        return image, label

#### Using albumentations to perform some image manipulations to make our model generalize better. We will use these transforms later while making the datasets.

In [ ]:
train_transforms = A.Compose(
    [
        A.Resize(height=160, width=240),
        A.Rotate(limit=35, p=1.0),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.1),
        A.Normalize(
            mean=[0.0, 0.0, 0.0],
            std=[1.0, 1.0, 1.0],
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ],
)

val_transforms = A.Compose(
    [
        A.Resize(height=160, width=240),
        A.Normalize(
            mean=[0.0, 0.0, 0.0],
            std=[1.0, 1.0, 1.0],
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ],
)

#### The train, test and validation datasets can be created as shown below.

In [ ]:
train_ds = FishDataset(images=train_data,labels=train_labels,transform=train_transforms)

val_ds = FishDataset(images=val_data,labels=val_labels,transform=val_transforms)

test_ds = FishDataset(images=test_data,labels=test_labels,transform=val_transforms)

#### We will convert these datasets into dataloaders which will pass image samples in “minibatches" and reshuffle the data at every epoch to reduce model overfitting.

In [ ]:
BATCH_SIZE = 128

train_loader = DataLoader(train_ds,batch_size=BATCH_SIZE,num_workers=4,pin_memory=True,shuffle=True)   

val_loader = DataLoader(val_ds,batch_size=BATCH_SIZE,num_workers=4,pin_memory=True)

test_loader = DataLoader(test_ds,batch_size=BATCH_SIZE,num_workers=4,pin_memory=True)

# Creating Our Model

#### We will now create a `FishModel()` class which will import the pretrained model for us and create a working model for our dataset. It also has a bunch of helper functions.

In [ ]:
class FishModel(nn.Module):
    
    def __init__(self, num_classes, pretrained=True):
        super().__init__()
        self.network = models.resnet34(pretrained=pretrained)
        self.network.fc = nn.Linear(self.network.fc.in_features, num_classes)

    def forward(self, xb):
        return self.network(xb)

    def training_step(self, batch):
        images, labels = batch
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels)  # Calculate loss
        return loss

    def validation_step(self, batch):
        images, labels = batch
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}

    def epoch_end(self, epoch, result):
        print("Epoch [{}],{} train_loss: {}, val_loss: {}, val_acc: {}".format(
            epoch, "last_lr: {:.5f},".format(result['lrs'][-1]) if 'lrs' in result else '', 
            result['train_loss'], result['val_loss'], result['val_acc']))



#### Here I created more helper functions which will be useful in fitting the data to the model and then evaluating the model performance afterwards. The `fit_one_cycle()` function takes in the number of epochs, our model, our dataloaders and optimization functions.

In [ ]:
def accuracy(outputs, labels):
        _, preds = torch.max(outputs, dim=1)
        return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [ ]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def fit_one_cycle(epochs, max_lr, model, train_loader, val_loader,
                  weight_decay=0, grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []

    # Set up custom optimizer with weight decay
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    # Set up one-cycle learning rate scheduler
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, steps_per_epoch=len(train_loader))

    for epoch in range(epochs):
        # Training Phase
        model.train()
        train_losses = []
        lrs = []
        for batch in tqdm(train_loader):
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()

            # Gradient clipping
            if grad_clip:
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)

            optimizer.step()
            optimizer.zero_grad()

            # Record & update learning rate
            lrs.append(get_lr(optimizer))
            sched.step()

        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)
    return history

# Training on GPU

#### We will now check the availability of GPU and if available move the data and the model on it. These functions will be useful in doing so.

In [ ]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')


def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)


class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""

    def __init__(self, dl, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl:
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)


#### To check the availability of the GPU

In [ ]:
device = get_default_device()
device

#### Now, as the GPU is available, we will move the data on it.

In [ ]:
train_dl = DeviceDataLoader(train_loader, device)
valid_dl = DeviceDataLoader(val_loader, device)

#### After loading the data on the GPU we will send our model on the GPU after instantiating it from our `FishModel()` class. The `len(classes)` is the number of classes which will generate a linear layer for 9 classes at the end of our pretrained model.

In [ ]:
model = FishModel(len(classes))
to_device(model, device);

#### We will now check for the accuracy and losses by the model without any training.

In [ ]:
history = [evaluate(model, valid_dl)]
history

# Model Training
#### Mentioning some hyperparameters before calling the fit_one_cycle() function.

In [ ]:
epochs = 6
max_lr = 0.01
grad_clip = 0.1
weight_decay = 1e-4
opt_func = torch.optim.Adam

#### Begin training...

In [ ]:
%%time
history += fit_one_cycle(epochs, max_lr, model, train_dl, valid_dl, 
                         grad_clip=grad_clip, 
                         weight_decay=weight_decay, 
                         opt_func=opt_func)

# Model Testing

#### Now that our model is trained we will start with the testing and evaluation of our model.

In [ ]:
def eval_accuracy(loader):  
    model.eval()
    corrects = 0
    total = 0
    for images, labels in loader:
        images, labels = to_device(images, device), to_device(labels, device)
        predictions = model(images)
        predict = to_device(torch.max(predictions.data, 1)[1], device)
        total += len(labels)
        corrects += (predict == labels).sum()
    accuracy = 100 * corrects / float(total)
    return accuracy

In [ ]:
test_acc = eval_accuracy(test_loader)
print(f' Accuracy on test images: {test_acc}')

#### The function `predict_image()` takes in test images, sends it to the device (i.e. GPU) and let our model predict it. This will generate a list of probabilities for each class and the function returns the class with the highest probability.

In [ ]:
def predict_image(image):
    xb = to_device(image.unsqueeze(0), device)
    yb = model(xb)
    _, preds  = torch.max(yb, dim=1)
    return classes[preds[0].item()]

#### Testing some random images individually.

In [ ]:
img, label = test_ds[100]
plt.imshow(img.permute(1, 2, 0))
print('Label:', classes[label], ', Predicted:', predict_image(img))


In [ ]:
img, label = test_ds[500]
plt.imshow(img.permute(1, 2, 0))
print('Label:', classes[label], ', Predicted:', predict_image(img))

In [ ]:
img, label = test_ds[800]
plt.imshow(img.permute(1, 2, 0))
print('Label:', classes[label], ', Predicted:', predict_image(img))

#  So how well did our model perform?

In [ ]:
def plot_losses(history):
    losses = [x['val_loss'] for x in history]
    plt.plot(losses, '-x')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.title('Loss vs. No. of epochs');

def plot_accuracies(history):
    accuracies = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs. No. of epochs');

In [ ]:
plot_losses(history)

In [ ]:
plot_accuracies(history)

In [ ]:
print("We used a RESNET34 model to predict and classify fish images and achieved a test accuracy of {:.2f}%.".format(test_acc))



#### We can try to further improve this by changing the batch size, using a different optimization algorithm, increasing the number of epochs and/or using different pretrained model such as RESNET50.